# Vegetation & Water Indices
Is it Green? Is it Wet? Calculating and visualizing Normalized Difference Vegetation Index (NDVI) & Normalized Difference Water Index (NDWI) from satellite imagery.

In [ ]:
import folium
import ee
import geemap
from datetime import datetime, timedelta

# Authenticate your Google account with Earth Engine.
ee.Authenticate()

# Write your project ID here, in quotes
ee.Initialize(project = "emerge-lessons")

In [12]:
def add_ee_layer(self, ee_image_object, vis_params, name):
    """Adds a method for displaying Earth Engine image tiles to folium map."""
    map_id_dict = ee.Image(ee_image_object).getMapId(vis_params)
    folium.raster_layers.TileLayer(
        tiles=map_id_dict['tile_fetcher'].url_format,
        attr='Map Data &copy; <a href="https://earthengine.google.com/">Google Earth Engine</a>',
        name=name,
        overlay=True,
        control=True
    ).add_to(self)

folium.Map.add_ee_layer = add_ee_layer

Draw a circle around an input point, which will be used to clip the satellite image.

In [2]:
# Region of interest
point = ee.Geometry.Point(-81.660044, 28.473813)
region = point.buffer(distance=100000)

Get Sentinel-2 satellite images that match our AOI and date range, with less than 10% cloud cover

In [3]:
collection = ee.ImageCollection('COPERNICUS/S2_SR_HARMONIZED') \
    .filterBounds(region) \
    .filterDate('2024-04-01', '2024-06-01') \
    .filter(ee.Filter.lte('CLOUDY_PIXEL_PERCENTAGE', 10))

Take the median value of the images in our collection; this helps remove clouds and get a clearer image

In [4]:
image = collection.median().clip(region)

Calculate NDVI (vegetation index) from the image
This shows how healthy the plants are: high NDVI = more green plants
Formula: (NIR - RED) / (NIR + RED)

In [5]:
ndvi = image.normalizedDifference(['B8', 'B4']).rename('NDVI')

Calculate NDWI (water index) from the image
This shows where the water is: high NDWI = more water
Formula: (GREEN - NIR) / (GREEN + NIR)

In [6]:
ndwi = image.normalizedDifference(['B3', 'B8']).rename('NDWI')

Set up an interactive map and center it over Florida.

In [21]:
map = folium.Map(location=[28.473813, -81.660044], tiles="Cartodb dark_matter", zoom_start=8)

Add the NDVI (vegetation) layer to the map, coloring high values green

In [22]:
map.add_ee_layer(ndvi, {'min': 0, 'max': 1, 'palette': ['white', 'green']}, 'NDVI')

Add the NDWI (water) layer to the map, coloring high values blue

In [23]:
map.add_ee_layer(ndwi, {'min': -1, 'max': 1, 'palette': ['white', 'blue']}, 'NDWI')

Show the map, with a layer control section in the top right to allow us to show or hide each layer.

In [24]:
folium.LayerControl(collapsed = False).add_to(map)
display(map)

In the interactive map above, click on the check box next to `NDWI` to hide the layer, showing `NDVI` below it. You can click the check box again to show `NDWI` again.